# Preproceeseing

In [1]:
from nba_api.stats.endpoints import PlayerGameLogs, LeagueStandingsV3
from nba_api.stats.static import players, teams

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)

In [2]:
def make_last (l1, df):
    cols = list(df.columns)
    for col in l1:
        cols.remove(col)
        cols.append(col)
    df = df[cols]
    return df


def add_target(group):
    group["target"] = group["won"].shift(-1)
    group["target"].fillna(2, inplace=True)
    return group


def check_data_quality(df):
    print("The shape:")
    print(df.shape)

    
    missing_values = df.isna().sum()
    if missing_values.sum() == 0:
        print("\nNo missing values found.")
    else:
        print("Columns with missing values:")
        print(missing_values[missing_values != 0])

    print("\nData types:")
    print(df.dtypes)
    
    print("\nUnique types:")
    print(df.nunique())

    print("\nSummary Statistics:")
    df.describe()

    df.hist(bins=50, figsize=(20,15))

In [3]:
def mergeDbStats(year, df):
    df_stats_new = PlayerGameLogs(season_nullable=year).get_data_frames()[0]
    df = pd.concat([df_stats_new, df], ignore_index=True)
    return df

    
    
    
df_stats_all_years = PlayerGameLogs(season_nullable='2016-17').get_data_frames()[0]
df_stats_all_years = mergeDbStats('2017-18', df_stats_all_years)
df_stats_all_years = mergeDbStats('2018-19', df_stats_all_years)
df_stats_all_years = mergeDbStats('2019-20', df_stats_all_years)
df_stats_all_years = mergeDbStats('2020-21', df_stats_all_years)
df_stats_all_years = mergeDbStats('2021-22', df_stats_all_years)
df_stats_all_years = mergeDbStats('2022-23', df_stats_all_years)

#check_data_quality(df_stats_all_years) 

In [4]:
df_player_preprocess = df_stats_all_years

In [5]:
df_player_preprocess["VIDEO_AVAILABLE_FLAG"] = df_player_preprocess['MATCHUP'].str.split(' ', n = 2, expand = True)[2]
df_player_preprocess.rename(columns={"VIDEO_AVAILABLE_FLAG": "team_opp"}, inplace=True)

del df_player_preprocess["MATCHUP"]


df_player_preprocess["GAME_DATE"] = df_player_preprocess['GAME_DATE'].str.split('T', n = 2, expand = True)[0]


WL = df_player_preprocess['WL']
del df_player_preprocess["WL"]
WL = WL.map({'W': 1, 'L': 0, np.nan: 2})
df_player_preprocess["won"] = WL


df_player_preprocess = df_player_preprocess.sort_values("GAME_DATE")


df_player_preprocess = df_player_preprocess.groupby("PLAYER_ID", group_keys=False).apply(add_target)


del df_player_preprocess['PLAYER_ID']
del df_player_preprocess['NICKNAME']
del df_player_preprocess['TEAM_NAME']
del df_player_preprocess['TEAM_ID']
del df_player_preprocess['GAME_ID']


df_player_preprocess = df_player_preprocess.rename(columns={'TEAM_ABBREVIATION': 'team'})
df_player_preprocess = df_player_preprocess.rename(columns={'PLAYER_NAME': 'player'})
df_player_preprocess = df_player_preprocess.rename(columns={'SEASON_YEAR': 'season'})


df_player_preprocess = make_last(list(['season', 'player', 'team', 'GAME_DATE']), df_player_preprocess)


print('WON columns value counts:')
print(df_player_preprocess["won"].value_counts())
print('\nTARGET column value counts:')
print(df_player_preprocess["target"].value_counts())

WON columns value counts:
1    85730
0    85415
Name: won, dtype: int64

TARGET column value counts:
1.0    85202
0.0    84782
2.0     1161
Name: target, dtype: int64


In [6]:
df_team_preprocess = pd.read_csv("nba_games.csv", index_col=0)
#check_data_quality(df_team_preprocess) 

In [7]:
df_team_preprocess = df_team_preprocess.sort_values("date")
df_team_preprocess = df_team_preprocess.reset_index(drop=True)


df_team_preprocess = df_team_preprocess.drop(["mp.1", "mp_opp.1", "index_opp"], axis=1)


df_team_preprocess = make_last(['team', 'home'], df_team_preprocess)


won = df_team_preprocess['won']
del df_team_preprocess["won"]
won = won.map({True: 1, False: 0, np.nan: 2})
df_team_preprocess["won"] = won


nulls = pd.isnull(df_team_preprocess).sum()
nulls = nulls[nulls > 0]
valid_columns = df_team_preprocess.columns[~df_team_preprocess.columns.isin(nulls.index)]
df_team_preprocess = df_team_preprocess[valid_columns]

In [8]:
df_player_preprocess.head()

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,team_opp,won,target,season,player,team,GAME_DATE
171144,12.000000,2,3,0.667,0,0,0.000,0,0,0.0,0,3,3,3,1,0,1,0,1,0,4,11,14.1,0,0,12.0,1,1,1,1,21657,15070,20108,2938,12221,18442,12221,13946,14843,13946,12923,9449,12475,5382,8857,12237,2482,1,5176,18615,18247,3615,16169,2032,118,17016,NYK,1,1.0,2016-17,DeAndre Liggins,CLE,2016-10-25
171092,20.866667,4,10,0.400,2,6,0.333,0,0,0.0,0,5,5,2,0,1,0,0,1,0,10,22,22.0,0,0,21.0,1,1,1,1,15215,8702,7506,14323,3010,2314,7723,13946,14843,13946,12923,4395,7038,8113,16648,4652,7994,1,5176,18615,10270,696,10424,2032,118,10056,GSW,1,1.0,2016-17,Manu Ginobili,SAS,2016-10-25
171093,3.683333,2,2,1.000,1,1,1.000,0,0,0.0,0,1,1,0,0,0,0,0,0,0,5,6,6.2,0,0,7.0,1,1,1,1,24907,15070,21916,1,6264,14538,1,13946,14843,13946,12923,17700,19668,18531,16648,12237,7994,1,1,18615,16895,6692,21924,2032,118,20990,GSW,1,1.0,2016-17,Davis Bertans,SAS,2016-10-25
171094,3.683333,1,2,0.500,0,0,0.000,0,1,0.0,0,0,0,0,0,0,0,0,0,1,2,-6,2.0,0,0,2.0,1,13060,13060,13060,24907,18863,21916,8149,12221,18442,12221,13946,13324,13946,12923,22443,23356,18531,16648,12237,7994,1,1,12086,21017,18336,24386,2032,118,24184,SAS,0,1.0,2016-17,JaVale McGee,GSW,2016-10-25
171095,28.400000,3,6,0.500,2,3,0.667,1,1,1.0,0,4,4,1,1,1,0,0,0,2,9,0,17.3,0,0,18.0,1,1,1,1,8758,11660,14148,8149,3010,7843,1998,10567,13324,1,12923,6518,9421,12386,8857,4652,7994,1,1,7572,11491,12096,13726,2032,118,12279,UTA,1,0.0,2016-17,Al-Farouq Aminu,POR,2016-10-25


In [9]:
df_team_preprocess.head()

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,total,mp_opp,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,team,home,won
0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,8.0,25.0,33.0,21.0,9.0,3.0,18.0,26.0,95.0,0.501,0.458,0.217,0.325,18.6,54.3,37.1,60.0,9.0,4.5,15.9,100.0,94.9,110.9,7.0,20.0,1.000,2.0,7.0,0.667,10.0,15.0,1.0,3.0,5.0,6.0,9.0,3.0,3.0,5.0,5.0,18.0,5.0,1.000,1.000,1.000,0.750,19.8,26.3,13.6,43.4,5.3,6.3,37.5,38.9,201.0,120.0,95,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,22.0,0.909,21.0,35.0,56.0,29.0,8.0,7.0,20.0,29.0,111.0,0.525,0.474,0.313,0.229,45.7,81.4,62.9,70.7,8.0,10.8,15.9,100.0,110.9,94.9,14.0,26.0,1.000,5.0,12.0,1.000,7.0,7.0,1.0,5.0,5.0,9.0,7.0,2.0,2.0,7.0,5.0,40.0,20.0,1.136,1.000,0.462,0.571,39.0,27.2,25.2,50.1,19.1,7.9,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,NOP,0,0
1,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,39.0,50.0,26.0,5.0,7.0,10.0,21.0,95.0,0.468,0.452,0.309,0.181,21.6,84.8,51.5,68.4,5.0,10.3,9.0,100.0,95.5,97.5,12.0,22.0,0.571,3.0,7.0,0.500,3.0,4.0,1.0,4.0,10.0,12.0,7.0,1.0,2.0,3.0,4.0,25.0,9.0,0.714,0.714,1.000,2.000,10.8,41.9,23.8,31.2,2.8,18.5,30.4,29.0,138.0,105.0,95,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,40.0,47.0,13.0,6.0,10.0,13.0,22.0,97.0,0.499,0.466,0.218,0.264,15.2,78.4,48.5,35.1,6.0,15.4,11.8,100.0,97.5,95.5,8.0,22.0,0.750,3.0,5.0,0.750,5.0,5.0,1.0,2.0,8.0,10.0,5.0,2.0,6.0,4.0,6.0,19.0,9.0,0.820,0.875,0.556,1.333,12.2,38.5,26.0,30.3,2.8,14.0,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,CLE,0,0
2,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,40.0,47.0,13.0,6.0,10.0,13.0,22.0,97.0,0.499,0.466,0.218,0.264,15.2,78.4,48.5,35.1,6.0,15.4,11.8,100.0,97.5,95.5,8.0,22.0,0.750,3.0,5.0,0.750,5.0,5.0,1.0,2.0,8.0,10.0,5.0,2.0,6.0,4.0,6.0,19.0,9.0,0.820,0.875,0.556,1.333,12.2,38.5,26.0,30.3,2.8,14.0,53.2,34.6,162.0,104.0,97,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,39.0,50.0,26.0,5.0,7.0,10.0,21.0,95.0,0.468,0.452,0.309,0.181,21.6,84.8,51.5,68.4,5.0,10.3,9.0,100.0,95.5,97.5,12.0,22.0,0.571,3.0,7.0,0.500,3.0,4.0,1.0,4.0,10.0,12.0,7.0,1.0,2.0,3.0,4.0,25.0,9.0,0.714,0.714,1.000,2.000,10.8,41.9,23.8,31.2,2.8,18.5,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,CHI,1,1
3,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,22.0,0.909,21.0,35.0,56.0,29.0,8.0,7.0,20.0,29.0,111.0,0.525,0.474,0.313,0.229,45.7,81.4,62.9,70.7,8.0,10.8,15.9,100.0,110.9,94.9,14.0,26.0,1.000,5.0,12.0,1.000,7.0,7.0,1.0,5.0,5.0,9.0,7.0,2.0,2.0,7.0,5.0,40.0,20.0,1.136,1.000,0.462,0.571,39.0,27.2,25.2,50.1,19.1,7.9,69.4,43.7,206.0,104.0,111,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,8.0,25.0,33.0,21.0,9.0,3.0,18.0,26.0,95.0,0.501,0.458,0.217,0.325,18.6,54.3,37.1,60.0,9.0,4.5,15.9,100.0,94.9,110.9,7.0,20.0,1.000,2.0,7.0,0.667,10.0,15.0,1.0,3.0,5.0,6.0,9.0,3.0,3.0,5.0,5.0,18.0,5.0,1.000,1.000,1.000,0.750,19.8,26.3,13.6,43.4,5.3,6.3,37.5,38.9,201.0,120.0,NOP,95,0,2016,2015-10-27,GSW,1,1
4,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,33.0,40.

In [10]:
df_player = df_player_preprocess
df_team = df_team_preprocess

In [11]:
removed_columns_player = ['season', 'player', 'team', 'GAME_DATE','team_opp', 'won', 'target']
selected_columns_player = df_player.columns[~df_player.columns.isin(removed_columns_player)]
scaler = MinMaxScaler()
df_player[selected_columns_player] = scaler.fit_transform(df_player[selected_columns_player])

In [12]:
removed_columns_team = ["season", "date", "won", "target", "team", "team_opp"]
selected_columns_team = df_team.columns[~df_team.columns.isin(removed_columns_team)]
scaler = MinMaxScaler()
df_team[selected_columns_team] = scaler.fit_transform(df_team[selected_columns_team])

In [13]:
df_player_rolling = df_player[list(selected_columns_player) + ["season","player", "won","team"]]
df_team_rolling = df_team[list(selected_columns_team) + ["won", "team", "season"]]

In [14]:
def find_team_averages(player):
    rolling = player.rolling(10).mean()
    return rolling

df_player_rolling = df_player_rolling.groupby(["player", "season"], group_keys=False).apply(find_team_averages)
df_team_rolling = df_team_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

In [15]:
rolling_cols_player = [f"{col}_10" for col in df_player_rolling.columns]
df_player_rolling.columns = rolling_cols_player
df_player = pd.concat([df_player, df_player_rolling], axis=1)


rolling_cols_team = [f"{col}_10" for col in df_team_rolling.columns]
df_team_rolling.columns = rolling_cols_team
df_team = pd.concat([df_team, df_team_rolling], axis=1)

In [16]:
df_player = df_player.dropna()
df_team = df_team.dropna()

In [17]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col


def add_col_player(df, col_name):
    return df.groupby("player", group_keys=False).apply(lambda x: shift_col(x, col_name))

def add_col_team(df, col_name):
    return df.groupby("team", group_keys=False).apply(lambda x: shift_col(x, col_name))

In [18]:
df_player["team_opp_next"] = add_col_player(df_player, "team_opp")
df_player["game_date_next"] = add_col_player(df_player, "GAME_DATE")

df_team["home_next"] = add_col_team(df_team, "home")
df_team["team_opp_next"] = add_col_team(df_team, "team_opp")
df_team["date_next"] = add_col_team(df_team, "date")

In [19]:
removed_columns_player = ['season', 'GAME_DATE', 'player', 'team', 'team_opp', 'game_date_next','team_opp', 'won', 'target']
removed_columns_team = ['season', 'date', 'home', 'team', 'home', 'team_opp','home_opp','home_next', 'team_opp_next', 'won']



df_player = make_last(removed_columns_player, df_player)
df_team = make_last(removed_columns_team, df_team)

In [20]:
df_player[df_player['player'] == 'Deni Avdija']

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,MIN_10,FGM_10,FGA_10,FG_PCT_10,FG3M_10,FG3A_10,FG3_PCT_10,FTM_10,FTA_10,FT_PCT_10,OREB_10,DREB_10,REB_10,AST_10,TOV_10,STL_10,BLK_10,BLKA_10,PF_10,PFD_10,PTS_10,PLUS_MINUS_10,NBA_FANTASY_PTS_10,DD2_10,TD3_10,WNBA_FANTASY_PTS_10,GP_RANK_10,W_RANK_10,L_RANK_10,W_PCT_RANK_10,MIN_RANK_10,FGM_RANK_10,FGA_RANK_10,FG_PCT_RANK_10,FG3M_RANK_10,FG3A_RANK_10,FG3_PCT_RANK_10,FTM_RANK_10,FTA_RANK_10,FT_PCT_RANK_10,OREB_RANK_10,DREB_RANK_10,REB_RANK_10,AST_RANK_10,TOV_RANK_10,STL_RANK_10,BLK_RANK_10,BLKA_RANK_10,PF_RANK_10,PFD_RANK_10,PTS_RANK_10,PLUS_MINUS_RANK_10,NBA_FANTASY_PTS_RANK_10,DD2_RANK_10,TD3_RANK_10,WNBA_FANTASY_PTS_RANK_10,won_10,team_opp_next,season,GAME_DATE,player,team,game_date_next,team_opp,won,target
67509,0.526155,0.304348,0.340909,0.467,0.357143,0.375000,0.556,0.038462,0.034483,1.0,0.000000,0.217391,0.166667,0.20,0.000000,0.2,0.0,0.250,0.333333,0.052632,0.281690,0.460177,0.394470,0.0,0.0,0.393939,0.0,0.885823,0.885823,0.885823,0.203658,0.140207,0.117373,0.470581,0.029021,0.045388,0.240039,0.640575,0.757247,0.000000,0.845710,0.180837,0.275411,0.133886,0.851180,0.098242,0.870127,0.801439,0.418030,0.544710,0.131699,0.553753,0.104415,0.783715,1.00000,0.094475,0.390860,0.108696,0.127273,0.4552,0.114286,0.145833,0.4456,0.019231,0.027586,0.3000,0.023077,0.156522,0.130000,0.092,0.041667,0.11,0.03,0.0375,0.433333,0.042105,0.100000,0.503540,0.204885,0.0,0.0,0.178788,0.0,0.708659,0.708659,0.708659,0.434729,0.559053,0.556203,0.478197,0.478568,0.437703,0.433272,0.738227,0.707879,0.641436,0.735667,0.377583,0.433322,0.428503,0.652447,0.445100,0.691285,0.138756,0.511910,0.653224,0.570002,0.447548,0.470742,0.783715,1.00000,0.487948,0.2,PHX,2020-21,2021-01-09,Deni Avdija,WAS,2021-01-11,MIA,0,1.0
67091,0.329249,0.130435,0.136364,0.500,0.000000,0.000000,0.000,0.000000,0.000000,0.0,0.076923,0.173913,0.166667,0.04,0.166667,0.0,0.0,0.000,0.500000,0.000000,0.084507,0.522124,0.136406,0.0,0.0,0.121212,0.0,0.000000,0.000000,0.000000,0.548686,0.467781,0.508230,0.331642,0.884688,0.874047,0.884688,0.846696,0.847806,0.846696,0.397861,0.272140,0.275411,0.605026,0.216660,0.867980,0.870127,0.000000,0.622530,0.846581,0.593434,0.311309,0.626990,0.783715,1.00000,0.611878,0.377041,0.113043,0.136364,0.4052,0.100000,0.137500,0.3456,0.015385,0.020690,0.2500,0.030769,0.156522,0.133333,0.092,0.050000,0.10,0.03,0.0375,0.433333,0.021053,0.098592,0.493805,0.201382,0.0,0.0,0.174747,0.0,0.620076,0.620076,0.620076,0.458851,0.546174,0.530900,0.511361,0.539238,0.467339,0.521741,0.758839,0.748484,0.659396,0.690882,0.377583,0.423673,0.428503,0.628741,0.499088,0.691285,0.138756,0.511910,0.725832,0.574031,0.469653,0.479645,0.783715,1.00000,0.498268,0.3,BKN,2020-21,2021-01-11,Deni Avdija,WAS,2021-01-31,PHX,1,1.0
64233,0.260860,0.043478,0.090909,0.250,0.000000,0.083333,0.000,0.000000,0.000000,0.0,0.076923,0.043478,0.066667,0.00,0.000000,0.0,0.0,0.000,0.333333,0.000000,0.028169,0.415929,0.070968,0.0,0.0,0.040404,0.0,0.000000,0.000000,0.000000,0.653250,0.730609,0.637121,0.775143,0.884688,0.577682,0.884688,0.846696,0.847806,0.846696,0.397861,0.706813,0.612763,0.871399,0.851180,0.867980,0.870127,0.000000,0.418030,0.846581,0.795694,0.693779,0.773416,0.783715,1.00000,0.796290,0.378732,0.108696,0.140909,0.3302,0.092857,0.141667,0.2456,0.015385,0.017241,0.2500,0.038462,0.147826,0.130000,0.084,0.050000,0.10,0.03,0.0375,0.400000,0.015789,0.094366,0.492035,0.194747,0.0,0.0,0.167677,0.0,0.531494,0.531494,0.531494,0.456598,0.559578,0.518485,0.588875,0.575905,0.453513,0.610210,0.758839,0.757540,0.659396,0.646097,0.409205,0.436594,0.4

In [21]:
df_team[df_team['team'] == 'WAS']

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,total,mp_opp,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,mp_10,fg_10,fga_10,fg%_10,3p_10,3pa_10,3p%_10,ft_10,fta_10,ft%_10,orb_10,drb_10,trb_10,ast_10,stl_10,blk_10,tov_10,pf_10,pts_10,ts%_10,efg%_10,3par_10,ftr_10,orb%_10,drb%_10,trb%_10,ast%_10,stl%_10,blk%_10,tov%_10,usg%_10,ortg_10,drtg_10,fg_max_10,fga_max_10,fg%_max_10,3p_max_10,3pa_max_10,3p%_max_10,ft_max_10,fta_max_10,ft%_max_10,orb_max_10,drb_max_10,trb_max_10,ast_max_10,stl_max_10,blk_max_10,tov_max_10,pf_max_10,pts_max_10,+/-_max_10,ts%_max_10,efg%_max_10,3par_max_10,ftr_max_10,orb%_max_10,drb%_max_10,trb%_max_10,ast%_max_10,stl%_max_10,blk%_max_10,tov%_max_10,usg%_max_10,ortg_max_10,drtg_max_10,total_10,mp_opp_10,fg_opp_10,fga_opp_10,fg%_opp_10,3p_opp_10,3pa_opp_10,3p%_opp_10,ft_opp_10,fta_opp_10,ft%_opp_10,orb_opp_10,drb_opp_10,trb_opp_10,ast_opp_10,stl_opp_10,blk_opp_10,tov_opp_10,pf_opp_10,pts_opp_10,ts%_opp_10,efg%_opp_10,3par_opp_10,ftr_opp_10,orb%_opp_10,drb%_opp_10,trb%_opp_10,ast%_opp_10,stl%_opp_10,blk%_opp_10,tov%_opp_10,usg%_opp_10,ortg_opp_10,drtg_opp_10,fg_max_opp_10,fga_max_opp_10,fg%_max_opp_10,3p_max_opp_10,3pa_max_opp_10,3p%_max_opp_10,ft_max_opp_10,fta_max_opp_10,ft%_max_opp_10,orb_max_opp_10,drb_max_opp_10,trb_max_opp_10,ast_max_opp_10,stl_max_opp_10,blk_max_opp_10,tov_max_opp_10,pf_max_opp_10,pts_max_opp_10,+/-_max_opp_10,ts%_max_opp_10,efg%_max_opp_10,3par_max_opp_10,ftr_max_opp_10,orb%_max_opp_10,drb%_max_opp_10,trb%_max_opp_10,ast%_max_opp_10,stl%_max_opp_10,blk%_max_opp_10,tov%_max_opp_10,usg%_max_opp_10,ortg_max_opp_10,drtg_max_opp_10,total_opp_10,home_opp_10,home_10,won_10,season_10,date_next,season,date,team,home,team_opp,home_opp,home_next,team_opp_next,won
376,0.0,0.431818,0.264706,0.521531,0.241379,0.227273,0.437055,0.302326,0.301587,0.649942,0.321429,0.425,0.408163,0.348837,0.454545,0.00,0.344828,0.361111,0.317308,0.480167,0.483301,0.310937,0.307305,0.440000,0.628571,0.567568,0.525036,0.516129,0.000000,0.380597,0.0,0.444566,0.419319,0.285714,0.095238,0.846154,0.142857,0.130435,1.000,0.28,0.257143,1.000000,0.307692,0.20,0.153846,0.304348,0.2,0.000000,0.181818,0.75,0.133333,0.560000,0.514,0.538319,0.50000,0.138735,0.192,0.241916,0.106711,0.561927,0.048,0.000,0.176101,0.059050,0.407583,0.388235,0.317308,0.0,0.386364,0.279412,0.447368,0.275862,0.333333,0.365796,0.325581,0.317460,0.666278,0.285714,0.350,0.326531,0.302326,0.136364,0.10,0.413793,0.305556,0.298077,0.436326,0.432220,0.443750,0.319899,0.371429,0.560000,0.432432,0.492132,0.156682,0.088312,0.440299,0.0,0.419319,0.444566,0.238095,0.238095,0.538462,0.285714,0.260870,0.571,0.12,0.085714,1.0,0.153846,0.50,0.346154,0.304348,0.1,0.181818,0.454545,0.50,0.166667,0.480000,0.265,0.307479,0.58375,0.043063,0.204,0.318563,0.141914,0.511468,0.017,0.044,0.213836,0.204108,0.270142,0.447059,0.298077,0.0,0.418182,0.327941,0.453828,0.293103,0.298485,0.431116,0.448837,0.411111,0.709802,0.321429,0.4300,0.412245,0.369767,0.400000,0.225,0.500000,0.425000,0.380769,0.469520,0.4

In [22]:
full_player = df_player.merge(df_player[rolling_cols_player + ["team_opp_next", "GAME_DATE", "team"]], 
                left_on=["team", "GAME_DATE"], 
                right_on=["team_opp_next", "GAME_DATE"])

full_team = df_team.merge(df_team[rolling_cols_team + ["team_opp_next", "date_next", "team"]],
                left_on=["team", "date_next"],
                right_on=["team_opp_next", "date_next"])

In [23]:
removed_columns_player = list(full_player.columns[full_player.dtypes == "object"]) + removed_columns_player
removed_columns_team = list(full_team.columns[full_team.dtypes == "object"]) + removed_columns_team


full_player = make_last(list(full_player.columns[full_player.dtypes == "object"]), full_player)
full_team = make_last(list(full_team.columns[full_team.dtypes == "object"]), full_team)

In [24]:
full = pd.merge(full_player, full_team, left_on=['team_x', 'GAME_DATE'], right_on=['team_x', 'date'])

In [25]:
full = make_last(list(full.columns[full.dtypes == "object"]), full)
full = make_last(['target'], full)

full = full.dropna(axis=1)
full = full.dropna()
full = full.rename(columns={'season_10_y': 'season'})

# bug

In [26]:
a = df_player_preprocess[df_player_preprocess['player'] == 'Deni Avdija']
x = df_player[df_player['player'] == 'Deni Avdija']
y = full_player[full_player['player'] == 'Deni Avdija']
z = full[full['player'] == 'Deni Avdija']

print(x.shape)
print(y.shape)
print(z.shape)
print(a.shape)

print(a['GAME_DATE'].unique().shape)
print(x['GAME_DATE'].unique().shape)
print(y['GAME_DATE'].unique().shape)
print(z['GAME_DATE'].unique().shape)


(174, 122)
(931, 181)
(630, 600)
(201, 63)
(201,)
(174,)
(125,)
(89,)


# K-Means

In [27]:
removed_cols = (list(full.columns[full.dtypes == "object"]) + ['target'])
selected_cols = full.columns[~full.columns.isin(removed_cols)]

kmeans = KMeans(20)
kmeans.fit(full[selected_cols])
pd.DataFrame(kmeans.cluster_centers_, columns=selected_cols).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
MIN,0.545585,0.534675,0.303359,0.321288,0.531716,0.502738,0.359812,0.272703,0.279972,0.315113,0.306380,0.516934,0.513317,0.285049,0.321078,0.315694,0.300039,0.518804,0.541343,0.219102
FGM,0.296930,0.268526,0.092512,0.105504,0.299604,0.266289,0.138536,0.083877,0.082774,0.102099,0.107280,0.260627,0.268505,0.083208,0.105918,0.112168,0.102896,0.258345,0.328971,0.063498
FGA,0.334796,0.305131,0.121278,0.133059,0.308885,0.273715,0.151546,0.101907,0.111135,0.129382,0.125515,0.294484,0.278222,0.112230,0.135348,0.127626,0.118084,0.293274,0.335133,0.080125
FG_PCT,0.472221,0.466720,0.373141,0.393679,0.515667,0.520352,0.475252,0.392962,0.368559,0.392503,0.416446,0.469852,0.513560,0.372313,0.392853,0.434288,0.419567,0.469010,0.521884,0.331634
FG3M,0.133071,0.105745,0.040261,0.056066,0.128142,0.137533,0.079668,0.039540,0.040765,0.041128,0.057692,0.109738,0.112135,0.042336,0.053188,0.063553,0.046981,0.125713,0.153472,0.035348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
total_opp_10_y,0.459934,0.405121,0.404261,0.462110,0.429664,0.450841,0.460131,0.403486,0.452976,0.406974,0.452673,0.452874,0.402267,0.444231,0.454717,0.458311,0.406263,0.444228,0.461242,0.452264
home_opp_10_y,0.499867,0.492415,0.490767,0.504439,0.499242,0.506551,0.497444,0.501018,0.508926,0.504778,0.491114,0.511941,0.504450,0.501289,0.491734,0.504702,0.502809,0.499706,0.503065,0.508689
home_10_y,0.500133,0.507585,0.509233,0.495561,0.500758,0.493449,0.502556,0.498982,0.491074,0.495222,0.508886,0.488059,0.495550,0.498711,0.508266,0.495298,0.497191,0.500294,0.496935,0.491311
won_10_y_y,0.510775,0.502130,0.507158,0.514834,0.498966,0.501300,0.518720,0.489332,0.498225,0.499047,0.492036,0.496538,0.488900,0.511604,0.510716,0.506300,0.506438,0.518183,0.505453,0.507887


In [28]:
full['grade'] = kmeans.labels_
full['grade'].value_counts()

4     41673
1     37446
2     34797
16    33068
10    32410
9     31164
3     30613
15    30030
6     29925
7     29856
14    29471
0     29401
18    28552
5     28470
13    27697
8     27662
12    27351
19    26055
17    25161
11    24755
Name: grade, dtype: int64

In [29]:
def add_target2(group):
    group["target"] = group["grade"].shift(-1)
    group["target"].fillna(21, inplace=True)
    return group

In [30]:
full = full.groupby("player", group_keys=False).apply(add_target2)

In [31]:
temp = full[full['player'] == 'Deni Avdija']
temp[['player', 'date', 'team_x', 'grade', 'target']].head(30)

,player,date,team_x,grade,target
410811,Deni Avdija,2021-01-09,WAS,0,0.0
410812,Deni Avdija,2021-01-09,WAS,0,0.0
410813,Deni Avdija,2021-01-09,WAS,0,0.0
410814,Deni Avdija,2021-01-09,WAS,0,0.0
410815,Deni Avdija,2021-01-09,WAS,0,0.0
410816,Deni Avdija,2021-01-09,WAS,0,0.0
410817,Deni Avdija,2021-01-09,WAS,0,0.0
410818,Deni Avdija,2021-01-09,WAS,0,6.0
421282,Deni Avdija,2021-01-31,WAS,6,3.0
422320,Deni Avdija,2021-02-02,WAS,3,6.0


# Start ML

In [32]:
from sklearn.linear_model import RidgeClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit


def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []
    
    seasons = sorted(data["season"].unique())
    
    for i in range(start, len(seasons), step):
        season = seasons[i]
        train = data[data["season"] < season]
        test = data[data["season"] == season]
        
        model.fit(train[predictors], train["target"])
        
        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["target"], preds], axis=1)
        combined.columns = ["actual", "prediction"]
        
        all_predictions.append(combined)
    return pd.concat(all_predictions)



rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(rr,
                                n_features_to_select=30, 
                                direction="forward",
                                cv=split,
                                n_jobs=1
                               )

# Improve the moudle

In [40]:
selected_columns = full.columns[~full.columns.isin((list(full.columns[full.dtypes == "object"]) + ['target'] + ['season']))]
selected_columns

Index(['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT',
       ...
       'blk%_max_opp_10_y', 'tov%_max_opp_10_y', 'usg%_max_opp_10_y',
       'ortg_max_opp_10_y', 'drtg_max_opp_10_y', 'total_opp_10_y',
       'home_opp_10_y', 'home_10_y', 'won_10_y_y', 'grade'],
      dtype='object', length=586)

In [34]:
full[selected_columns]

,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,MIN_10_x,FGM_10_x,FGA_10_x,FG_PCT_10_x,FG3M_10_x,FG3A_10_x,FG3_PCT_10_x,FTM_10_x,FTA_10_x,FT_PCT_10_x,OREB_10_x,DREB_10_x,REB_10_x,AST_10_x,TOV_10_x,STL_10_x,BLK_10_x,BLKA_10_x,PF_10_x,PFD_10_x,PTS_10_x,PLUS_MINUS_10_x,NBA_FANTASY_PTS_10_x,DD2_10_x,TD3_10_x,WNBA_FANTASY_PTS_10_x,GP_RANK_10_x,W_RANK_10_x,L_RANK_10_x,W_PCT_RANK_10_x,MIN_RANK_10_x,FGM_RANK_10_x,FGA_RANK_10_x,FG_PCT_RANK_10_x,FG3M_RANK_10_x,FG3A_RANK_10_x,FG3_PCT_RANK_10_x,FTM_RANK_10_x,FTA_RANK_10_x,FT_PCT_RANK_10_x,OREB_RANK_10_x,DREB_RANK_10_x,REB_RANK_10_x,AST_RANK_10_x,TOV_RANK_10_x,STL_RANK_10_x,BLK_RANK_10_x,BLKA_RANK_10_x,PF_RANK_10_x,PFD_RANK_10_x,PTS_RANK_10_x,PLUS_MINUS_RANK_10_x,NBA_FANTASY_PTS_RANK_10_x,DD2_RANK_10_x,TD3_RANK_10_x,WNBA_FANTASY_PTS_RANK_10_x,won_10_x_x,won_x,MIN_10_y,FGM_10_y,FGA_10_y,FG_PCT_10_y,FG3M_10_y,FG3A_10_y,FG3_PCT_10_y,FTM_10_y,FTA_10_y,FT_PCT_10_y,OREB_10_y,DREB_10_y,REB_10_y,AST_10_y,TOV_10_y,STL_10_y,BLK_10_y,BLKA_10_y,PF_10_y,PFD_10_y,PTS_10_y,PLUS_MINUS_10_y,NBA_FANTASY_PTS_10_y,DD2_10_y,TD3_10_y,WNBA_FANTASY_PTS_10_y,GP_RANK_10_y,W_RANK_10_y,L_RANK_10_y,W_PCT_RANK_10_y,MIN_RANK_10_y,FGM_RANK_10_y,FGA_RANK_10_y,FG_PCT_RANK_10_y,FG3M_RANK_10_y,FG3A_RANK_10_y,FG3_PCT_RANK_10_y,FTM_RANK_10_y,FTA_RANK_10_y,FT_PCT_RANK_10_y,OREB_RANK_10_y,DREB_RANK_10_y,REB_RANK_10_y,AST_RANK_10_y,TOV_RANK_10_y,STL_RANK_10_y,BLK_RANK_10_y,BLKA_RANK_10_y,PF_RANK_10_y,PFD_RANK_10_y,PTS_RANK_10_y,PLUS_MINUS_RANK_10_y,NBA_FANTASY_PTS_RANK_10_y,DD2_RANK_10_y,TD3_RANK_10_y,WNBA_FANTASY_PTS_RANK_10_y,won_10_y_x,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,fg_max,fga_max,fg%_max,3p_max,3pa_max,3p%_max,ft_max,fta_max,ft%_max,orb_max,drb_max,trb_max,ast_max,stl_max,blk_max,tov_max,pf_max,pts_max,+/-_max,ts%_max,efg%_max,3par_max,ftr_max,orb%_max,drb%_max,trb%_max,ast%_max,stl%_max,blk%_max,tov%_max,usg%_max,ortg_max,drtg_max,total,mp_opp,fg_opp,fga_opp,fg%_opp,3p_opp,3pa_opp,3p%_opp,ft_opp,fta_opp,ft%_opp,orb_opp,drb_opp,trb_opp,ast_opp,stl_opp,blk_opp,tov_opp,pf_opp,pts_opp,ts%_opp,efg%_opp,3par_opp,ftr_opp,orb%_opp,drb%_opp,trb%_opp,ast%_opp,stl%_opp,blk%_opp,tov%_opp,usg%_opp,ortg_opp,drtg_opp,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,mp_10_x,fg_10_x,fga_10_x,fg%_10_x,3p_10_x,3pa_10_x,3p%_10_x,ft_10_x,fta_10_x,ft%_10_x,orb_10_x,drb_10_x,trb_10_x,ast_10_x,stl_10_x,blk_10_x,tov_10_x,pf_10_x,pts_10_x,ts%_10_x,efg%_10_x,3par_10_x,ftr_10_x,orb%_10_x,drb%_10_x,trb%_10_x,ast%_10_x,stl%_10_x,blk%_10_x,tov%_10_x,usg%_10_x,ortg_10_x,drtg_10_x,fg_max_10_x,fga_max_10_x,fg%_max_10_x,3p_max_10_x,3pa_max_10_x,3p%_max_10_x,ft_max_10_x,fta_max_10_x,ft%_max_10_x,orb_max_10_x,drb_max_10_x,trb_max_10_x,ast_max_10_x,stl_max_10_x,blk_max_10_x,tov_max_10_x,pf_max_10_x,pts_max_10_x,+/-_max_10_x,ts%_max_10_x,efg%_max_10_x,3par_max_10_x,ftr_max_10_x,orb%_max_10_x,drb%_max_10_x,trb%_max_10_x,ast%_max_10_x,stl%_max_10_x,blk%_max_10_x,tov%_max_10_x,usg%_max_10_x,ortg_max_10_x,drtg_max_10_x,total_10_x,mp_opp_10_x,fg_opp_10_x,fga_opp_10_x,fg%_opp_10_x,3p_opp_10_x,3pa_opp_10_x,3p%_opp_10_x,ft_opp_10_x,fta_opp_10_x,ft%_opp_10_x,orb_opp_10_x,drb_opp_10_x,trb_opp_10_x,ast

In [35]:
#sfs.fit(full[selected_columns], full["target"])
#predictors = list(selected_columns[sfs.get_support()])

selected_cols = ['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2', 'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK', 'L_RANK','W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK',
 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
 'WNBA_FANTASY_PTS_RANK',
 'target',
 'season']
predictions = backtest(full, rr,selected_cols)
accuracy_score(predictions["actual"], predictions["prediction"])

0.07589472105457302